In [4]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pickle
import sklearn.metrics as metrics
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
import matplotlib.pyplot as plt
from numpy import where

In [5]:
# infile = open('XGBoost_model.sav','rb')
# model = pickle.load(infile)
# infile.close()

In [6]:
flights = pd.read_csv('data/flights.csv')
flights = flights[['fl_date','mkt_carrier','origin','dest'
                  ,'taxi_out','taxi_in','cancelled','crs_elapsed_time','arr_delay']]

#features and target
X = flights.loc[:,flights.columns!='cancelled']
y = flights[['cancelled']]

In [7]:
X['fl_date'] = pd.to_datetime(X['fl_date'],format='%Y-%m-%d')
X['year'] = pd.DatetimeIndex(X['fl_date']).year
X['month'] = pd.DatetimeIndex(X['fl_date']).month
X['day'] = pd.DatetimeIndex(X['fl_date']).day

In [8]:
X = X[['month','day','origin','dest','crs_elapsed_time']]

In [9]:
X= pd.get_dummies(X,columns=['origin','dest'])

In [15]:
# #calculating the weights of class 0 to 1
# class_0 = y.loc[y['cancelled'] == 0].value_counts().values[0]
# class_1 = y.loc[y['cancelled'] == 1].value_counts().values[0]
# weight = class_0/class_1
# print('Ratio of Class 0 to Class 1:',weight)

In [12]:
oversample = SMOTE(random_state=0)
X_resampled, y_resampled = oversample.fit_sample(X, y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,shuffle=True, test_size=0.30, random_state=123)

In [14]:
model = xgb.XGBClassifier()
model.fit(X_resampled.values,y_resampled.values.reshape(1,-1)[0])
filename = 'XGBoost_model_smote.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
moldel.predic